In [1]:
!curl ipinfo.io

{
  "ip": "134.94.68.46",
  "hostname": "ibg4699.ibg.kfa-juelich.de",
  "city": "Jülich",
  "region": "North Rhine-Westphalia",
  "country": "DE",
  "loc": "50.9215,6.3627",
  "org": "AS680 Verein zur Foerderung eines Deutschen Forschungsnetzes e.V.",
  "postal": "52428",
  "timezone": "Europe/Berlin",
  "readme": "https://ipinfo.io/missingauth"
}

In [14]:
!nvidia-smi

Fri Apr 28 10:47:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.89.02    Driver Version: 528.49       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P620         On   | 00000000:01:00.0 Off |                  N/A |
| N/A   51C    P8    N/A /  N/A |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from platform import python_version

print(python_version())

3.9.16


In [2]:

!pip3 install git+https://github.com/huggingface/peft.git --quiet

In [1]:
# Standard library imports
import textwrap

# Third-party imports
from peft import PeftModel
import torch
from transformers import LlamaTokenizer , LlamaForCausalLM, GenerationConfig


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/xr/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda121_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary /home/xr/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda121_nocublaslt.so...


/home/xr/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /home/xr/miniconda3 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/xr/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/home/xr/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


In [4]:
# Check available GPUs for computation
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    # Print details of all available GPUs
    for i in range(num_gpus):
        gpu_props = torch.cuda.get_device_properties(i)
        print(f"Device details for GPU {i+1}:")
        print(f"* Name: {gpu_props.name}")
        print(f"* Memory size: {round(gpu_props.total_memory / 1024**3, 2)} GB")
        if i == num_gpus-1:
            continue
        else:
            print("-"*79)
    # Get the currently active GPU device and print its name and memory size
    active_gpu = torch.cuda.current_device()
    active_gpu_props = torch.cuda.get_device_properties(active_gpu)
    print("="*79)
    print(f"Currently active GPU device: {active_gpu_props.name}")
    print(f"Memory size: {round(active_gpu_props.total_memory / 1024**3, 2)} GB")
    print("="*79)
else:
    print("No GPU devices found.")

Device details for GPU 1:
* Name: Quadro P620
* Memory size: 4.0 GB
Currently active GPU device: Quadro P620
Memory size: 4.0 GB


In [7]:

# Choose which model to run
# base_model = "decapoda-research/llama-7b-hf" #@param ["decapoda-research/llama-7b-hf", "decapoda-research/llama-13b-hf", "decapoda-research/llama-30b-hf"]
base_model = "emeerladdev/alpaca-lora-dolly-2.0-3b" #@param ["tloen/alpaca-lora-7b", "chansung/alpaca-lora-13b", "chansung/gpt4-alpaca-lora-7b"]

# Load tokeniser, base model and fine-tuned model
tokeniser = LlamaTokenizer.from_pretrained(base_model)
model = LlamaForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    device_map="auto"
)


OSError: Can't load tokenizer for 'emeerladdev/alpaca-lora-dolly-2.0-3b'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'emeerladdev/alpaca-lora-dolly-2.0-3b' is the correct path to a directory containing all relevant files for a LlamaTokenizer tokenizer.

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftConfig, PeftModelForCausalLM

peft_model_id = 'GrantC/alpaca-opt-1.3b-lora'
BASE_MODEL = 'facebook/opt-1.3b'
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

model = PeftModelForCausalLM.from_pretrained(model, peft_model_id, device_map="auto")

prompt = "Write a blog post about shaving cream:"
print(prompt)
inputs = tokenizer(prompt, return_tensors='pt')
output = model.generate(input_ids=inputs["input_ids"], do_sample= True, penalty_alpha=0.6, top_k=4, max_new_tokens=256)
outputs = tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(outputs)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/xr/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda121_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary /home/xr/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda121_nocublaslt.so...


/home/xr/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /home/xr/miniconda3 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/xr/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/home/xr/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


Write a blog post about shaving cream:
Write a blog post about shaving cream:  1. What is it? 2. How does it work? 3. How does it compare to other shaving creams? 4. How does it smell? 5. How does it feel when you apply it?
I'll do it tomorrow.  It's like shaving gel but with a lot of alcohol in it and it's not as smooth.  It's a little sticky too.  I don't really like it.  I'm going to get some aftershave and try it again.
I don't think you should shave with a lot of alcohol. I don't know how to shave without alcohol, but if it's that bad, I'm guessing you don't shave often enough to get used to the smell of alcohol.
I shave everyday.  I'm going to shave with aftershave.
You should try shaving without alcohol. It'll probably make the shave smoother, but it'll also probably make the smell of alcohol a little stronger.


In [3]:
prompt = "what is a data management plan"
print(prompt)
inputs = tokenizer(prompt, return_tensors='pt')
output = model.generate(input_ids=inputs["input_ids"], do_sample= True, penalty_alpha=0.6, top_k=4, max_new_tokens=256)
outputs = tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(outputs)

what is a data management plan
what is a data management plan?

Data is the lifeblood of the modern enterprise, but it's also one of the biggest risks for businesses.

The good news is that data is no longer just a problem for IT. It's a strategic issue for every business.

In this video, you’ll learn:

What data management is and how it can help your business

How data can help you manage your risk

Why data is a critical part of your business strategy

Why you need a data management plan

How to get started

What is a data management plan?

Data is a vital component for any enterprise, but it's a problem that's often overlooked by IT.

Data is a critical component for any enterprise, but it's a problem that's often overlooked by IT.

In this video, you'll learn:

Why data is a critical part of your business strategy

Why you need a data management plan

How to get started

What does a data management plan do?

Data management plans help you manage the risk of losing your data. They h